In [94]:
import pandas as pd
import statsmodels.formula.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

In [102]:
from experimentation import data_preparation
from experimentation import regression_OLS

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [103]:
df2 = regression_OLS(file_location = "uk_data.csv", splits = 2, lags = 2, train_share = 0.8, p_cutoff = 0.05)

/Users/martinlyngerasmussen/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1738: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss
/Users/martinlyngerasmussen/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1918: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/Users/martinlyngerasmussen/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  warnings.warn('covariance of constraints does not have full '
/Users/martinlyngerasmussen/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3

In [104]:
list_ = list(df1.keys())

In [105]:
df3 = df2[-1]

In [109]:
df3.head()

,y,split_1_y_fitted,split_1_y_pred,split_2_y_fitted,split_2_y_pred
date,,,,,
2005-01-31,0.876095,NaN,NaN,NaN,NaN
2005-02-28,0.000000,NaN,NaN,NaN,NaN
2005-03-31,-0.372208,-0.310691,NaN,NaN,NaN
2005-04-30,1.245330,0.198369,NaN,NaN,NaN
2005-05-31,-0.369004,0.117439,NaN,NaN,NaN
